# Cargar Datos

## 1. Contexto, librerías, rutas, carga y validaciones

Este notebook tiene como objetivo realizar la carga inicial del dataset utilizado en el proyecto, partiendo de un archivo no productivo proporcionado por la institución. 

En entornos productivos, la información suele provenir de un DWH o Datalake como resultado de procesos ETL previos. Sin embargo, para efectos académicos, se emplea un archivo local en formato Excel.

Las actividades principales de este notebook son:
- Cargar el dataset.
- Estandarizar nombres de columnas.
- Realizar validaciones básicas.
- Unificar representación de valores nulos.
- Ajustar tipos de datos.
- Guardar una versión procesada del dataset.

In [1]:
import pandas as pd

In [2]:
DATA_PATH = "../../Base_de_datos.xlsx"

In [3]:
df = pd.read_excel(DATA_PATH)

In [4]:
print("Dimensiones del dataset:", df.shape)
df.head()

Dimensiones del dataset: (10763, 23)


,tipo_credito,fecha_prestamo,capital_prestado,plazo_meses,edad_cliente,tipo_laboral,salario_cliente,total_otros_prestamos,cuota_pactada,puntaje,...,saldo_mora,saldo_total,saldo_principal,saldo_mora_codeudor,creditos_sectorFinanciero,creditos_sectorCooperativo,creditos_sectorReal,promedio_ingresos_datacredito,tendencia_ingresos,Pago_atiempo
0,7,2024-12-21 11:31:35,3692160.0,10,42,Independiente,8000000,2500000,341296,88.768094,...,0.0,51258.0,51258.0,0.0,5,0,0,908526.0,Estable,1
1,4,2025-04-22 09:47:35,840000.0,6,60,Empleado,3000000,2000000,124876,95.227787,...,0.0,8673.0,8673.0,0.0,0,0,2,939017.0,Creciente,1
2,9,2026-01-08 12:22:40,5974028.4,10,36,Independiente,4036000,829000,529554,47.613894,...,0.0,18702.0,18702.0,0.0,3,0,0,NaN,NaN,0
3,4,2025-08-04 12:04:10,1671240.0,6,48,Empleado,1524547,498000,252420,95.227787,...,0.0,15782.0,15782.0,0.0,3,0,0,1536193.0,Creciente,1
4,9,2025-04-26 11:24:26,2781636.0,11,44,Empleado,5000000,4000000,217037,95.227787,...,0.0,204804.0,204804.0,0.0,3,0,1,933473.0,Creciente,1


In [7]:
df.isnull().sum()

tipo_credito                        0
fecha_prestamo                      0
capital_prestado                    0
plazo_meses                         0
edad_cliente                        0
tipo_laboral                        0
salario_cliente                     0
total_otros_prestamos               0
cuota_pactada                       0
puntaje                             0
puntaje_datacredito                 6
cant_creditosvigentes               0
huella_consulta                     0
saldo_mora                        156
saldo_total                       156
saldo_principal                   405
saldo_mora_codeudor               590
creditos_sectorfinanciero           0
creditos_sectorcooperativo          0
creditos_sectorreal                 0
promedio_ingresos_datacredito    2930
tendencia_ingresos               2932
pago_atiempo                        0
dtype: int64

In [8]:
df.duplicated().sum()

0

## 2. Tratamiento de datos nulos

In [ ]:
df = df[df["puntaje_datacredito"].notna()]

In [10]:
saldo_cols = [
    "saldo_mora",
    "saldo_total",
    "saldo_principal",
    "saldo_mora_codeudor"
]

df[saldo_cols] = df[saldo_cols].fillna(0)

In [11]:
mediana_ingresos = df["promedio_ingresos_datacredito"].median()
df["promedio_ingresos_datacredito"] = df["promedio_ingresos_datacredito"].fillna(mediana_ingresos)

In [12]:
df["tendencia_ingresos"] = df["tendencia_ingresos"].fillna("Desconocido")

In [13]:
df.isnull().sum()

tipo_credito                     0
fecha_prestamo                   0
capital_prestado                 0
plazo_meses                      0
edad_cliente                     0
tipo_laboral                     0
salario_cliente                  0
total_otros_prestamos            0
cuota_pactada                    0
puntaje                          0
puntaje_datacredito              0
cant_creditosvigentes            0
huella_consulta                  0
saldo_mora                       0
saldo_total                      0
saldo_principal                  0
saldo_mora_codeudor              0
creditos_sectorfinanciero        0
creditos_sectorcooperativo       0
creditos_sectorreal              0
promedio_ingresos_datacredito    0
tendencia_ingresos               0
pago_atiempo                     0
dtype: int64

Se aplicaron tratamientos básicos de valores nulos:
- Eliminación de registros con ausencia de puntaje_datacredito.
- Imputación en cero para variables monetarias.
- Imputación con mediana para promedio de ingresos.
- Creación de categoría "Desconocido" para tendencia de ingresos.

In [14]:
# nuevo shape
print("Dimensiones del dataset después de la limpieza:", df.shape)

Dimensiones del dataset después de la limpieza: (10757, 23)


## 3. Normalización básica

In [5]:
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
)

In [6]:
df.columns

Index(['tipo_credito', 'fecha_prestamo', 'capital_prestado', 'plazo_meses',
       'edad_cliente', 'tipo_laboral', 'salario_cliente',
       'total_otros_prestamos', 'cuota_pactada', 'puntaje',
       'puntaje_datacredito', 'cant_creditosvigentes', 'huella_consulta',
       'saldo_mora', 'saldo_total', 'saldo_principal', 'saldo_mora_codeudor',
       'creditos_sectorfinanciero', 'creditos_sectorcooperativo',
       'creditos_sectorreal', 'promedio_ingresos_datacredito',
       'tendencia_ingresos', 'pago_atiempo'],
      dtype='object')

## 4. Conversión de tipos

In [ ]:
df["fecha_prestamo"] = pd.to_datetime(df["fecha_prestamo"]) # Convertir a tipo fecha

In [ ]:
cat_cols = [
    "tipo_credito",
    "tipo_laboral",
    "tendencia_ingresos"
]

df[cat_cols] = df[cat_cols].astype("category") # Convertir a tipo categoría

In [ ]:
df["pago_atiempo"] = df["pago_atiempo"].astype(int) # Convertir booleano a entero

In [18]:
df.dtypes

tipo_credito                           category
fecha_prestamo                   datetime64[ns]
capital_prestado                        float64
plazo_meses                               int64
edad_cliente                              int64
tipo_laboral                           category
salario_cliente                           int64
total_otros_prestamos                     int64
cuota_pactada                             int64
puntaje                                 float64
puntaje_datacredito                     float64
cant_creditosvigentes                     int64
huella_consulta                           int64
saldo_mora                              float64
saldo_total                             float64
saldo_principal                         float64
saldo_mora_codeudor                     float64
creditos_sectorfinanciero                 int64
creditos_sectorcooperativo                int64
creditos_sectorreal                       int64
promedio_ingresos_datacredito           